# Vent Validation

ADI instrumentation pressure is mmHg
Honeywell MPRLS (aka breath) is hPA


In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import sys
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

def save_text(txt, width, height):
    fig = plt.figure()
    fig.set_size_inches(width, height, forward = False)
    # fig.clf()
    fig.text(0.5,0.5,txt, transform=fig.transFigure, size=24, ha="center")
    save_fig(fig)
            
def save_fig(fig, title=""):
    if report:
        fig.suptitle(title, fontsize=16)
        report.savefig(fig)
    
    
def _plot(ndarray, components, offset, off, axs, legend, y_column, style='-'):
    ndkeys = ndarray.dtype.names
    comps = components if components else [ k for k in ndkeys if k != y_column]
    for c in comps:
        offset += off
        x = ndarray[y_column]
        y = ndarray[c] + offset
        axs.plot(x, y, style)
    axs.set_ylabel(f"{c}")
    return offset

    
def simple_plot(axs, streams, components=None, off=0, y_column='elapsed_s', title='', leg_contents=[], style='-'):

    if type(streams) != type([]):
        streams =[streams]
    
    offset = 0
    legend = []    
    for idx, stream in enumerate(streams):
        offset = _plot(stream, components, offset, off, axs, legend, y_column, style=style)
        if idx < len(leg_contents):
            legend.append(leg_contents[idx])
        
    axs.legend(legend, frameon=False, loc='lower right')
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.set_xlabel(f"{y_column}")
    axs.set_title(title)
    
    
        



In [ ]:

# 40psi
#recording = '../vent/02jun23pc/'
#offset = -0.2
"""
80, 0.02,   # ramp up                                                                                                                                                                                                                                                                         
100, 0.02,   # hold top                                                                                                                                                                                                                                                                       
75,  0.96,     # ramp down                                                                                                                                                                                                                                                                    
0,  2.0,   # hold bottom   
"""


# 40psi
#recording = '../vent/02jun23vc/'
#offset = 2.0
"""
80, 0.1,   # ramp up                                                                                                                                                                                                                                                                          
90, 0.9,   # hold top                                                                                                                                                                                                                                                                         
0,  0,     # ramp down                                                                                                                                                                                                                                                                        
0,  2.0,   # hold bottom       
"""

# 40psi
recording = '../vent/03jun00peep/'
offset = 2.0
"""
82, 0.1,   # ramp up                                                                                                                                                                                                                                                                          
90, 0.9,   # hold top                                                                                                                                                                                                                                                                         
82,  0,     # ramp down                                                                                                                                                                                                                                                                       
82,  2.0,   # hold bottom  
"""



# open a pdf report
report = PdfPages(recording + 'report.pdf')

## AD data

Read in data exported from labchart

In [ ]:
# read lab chart exported data
ad_df = pd.read_csv(recording+'labchart.txt', delimiter='\t', names=["ts", "flow", "pressure", "tidal", "minute"])
ad_df["flowmin"] = ad_df["flow"] * 60.0
ad_df["flowagg"] = ad_df["flowmin"]
ad_np = ad_df.to_records(index=False)

fig, axs = plt.subplots(ncols=1, nrows=4, constrained_layout=True)
fig.set_size_inches(15, 9, forward = False)
simple_plot(axs[0], [ad_np], components=["flow"], y_column="ts", leg_contents=["Flow (LPS)"], style="r-")
simple_plot(axs[1], [ad_np], components=["pressure"], y_column="ts", leg_contents=["Pressure (mmHg)"], style="b-")
simple_plot(axs[2], [ad_np], components=["tidal"], y_column="ts", leg_contents=["Tidal Volume (L)"], style="g-")
simple_plot(axs[3], [ad_np], components=["minute"], y_column="ts", leg_contents=["Minute Ventilation (L)"], style="m-")
save_fig(fig, title="Labchart Report")

In [ ]:
# read InVent exported data 
ci_df = pd.read_csv(recording+'sensors.out', delimiter=' ', names=["epoch","flowmin","in_p1","in_p2","in_flowmin","ex_p1","ex_p2","ex_flowmin"])
start_time = ci_df["epoch"].iloc[0]

# composite flow
#inflow = ci_df["flowmin"]
#exflow = ci_df["ex_flowmin"]
#flow = inflow
#for i,f in enumerate(flow):
#    if f < exflow[i]:
#        flow[i] = -exflow[i]

# inflow exflow and flow (composite) in LPS
ci_df["inflow"] = ci_df["in_flowmin"] / 60.0
ci_df["exflow"] = -ci_df["ex_flowmin"] / 60.0
ci_df["flow"] = ci_df["flowmin"] / 60

# convert pressure from PA to mmHg
ci_df["pressure"] = ci_df["ex_p1"] / 133 

ci_df["ts"] = ci_df["epoch"] - start_time + offset

ci_np = ci_df.to_records(index=False)

fig, axs = plt.subplots(ncols=1, nrows=2, constrained_layout=True)
fig.set_size_inches(15, 9, forward = False)
simple_plot(axs[0], [ci_np], components=["flow"], y_column="ts", leg_contents=["flow"], style="r-")
simple_plot(axs[1], [ci_np], components=["pressure"], y_column="ts", leg_contents=["pressure"], style="b-")
#save_fig(fig, title="InVent Inspiratory")

In [ ]:
in_df = ci_df.copy()
in_df["flow"] = in_df["inflow"]

ex_df = ci_df.copy()
ex_df["flow"] = in_df["exflow"]

fig, axs = plt.subplots(ncols=1, nrows=1, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)
simple_plot(axs, [ad_np, ci_np,], components=["flow"], y_column="ts", leg_contents=["AD Flow (LPS)","Our Flow (LPS)", "Our In Flow", "Our Ex Flow"])
save_fig(fig, title="Flow Validation")

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=1, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)
simple_plot(axs, [ad_np, ci_np], components=["pressure"], y_column="ts", leg_contents=["AD Pressure (mmHg)", "Our Pressure (mmHg)"])
save_fig(fig, title="Pressure Validation")


## Calculate Sample Rates

Vent should be at 75Hz
AD should be at 1kHz

In [ ]:
# calculate sample rates
ad_time = np.max(ad_df['ts'])-np.min(ad_df['ts'])
ci_time = np.max(ci_df['ts'])-np.min(ci_df['ts'])
print(f"AD samples {ad_np.shape[0]/ad_time}")
print(f"CI samples {ci_np.shape[0]/ci_time}")

## Pressure Differential

display the venturi differences for inspiration and expiration

In [ ]:
p1_df = ci_df.copy()
p1_df["pressure"] = p1_df["in_p1"]
p2_df = ci_df.copy()
p2_df["pressure"] = p2_df["in_p2"]

fig, axs = plt.subplots(ncols=1, nrows=1, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)
simple_plot(axs, [p1_df.to_records(index=False), p2_df.to_records(index=False)], components=["pressure"], y_column="ts", leg_contents=["p1","p2"])
#save_fig(fig, title="Venturi Pressures")

## Smoothing of pressure signals

can also be accomplished at the sensor

In [ ]:
def moving_avg_rms(data, window_size): # Calculate RMS
    # Moving avg RMS: https://stackoverflow.com/questions/8245687/numpy-root-mean-squared-rms-smoothing-of-a-signal
    data2 = np.power(data,2)
    window = np.ones(window_size)/float(window_size)
    return(np.sqrt(np.convolve(data2, window, 'valid')))

window = 10
f_df = ci_df.copy()
f_df["ex_flowmin"] = np.append(moving_avg_rms(ci_df["ex_flowmin"], window), np.zeros(window-1))

# inverse if on experiatory
#f_df["flowmin"] = -f_df["flowmin"]

fig, axs = plt.subplots(ncols=1, nrows=1, constrained_layout=True)
fig.set_size_inches(15, 10, forward = False)
simple_plot(axs, [ci_np,f_df.to_records(index=False)], components=["ex_flowmin"], y_column="ts", leg_contents=["f1","f2"])

In [ ]:
report.close()
report = None